In [2]:
!pip install nnfs

In [40]:
import numpy as np
import pandas as pd
import nnfs
from nnfs.datasets import vertical_data
import numpy as np
nnfs.init()
class Dense_Layer:
  def __init__(self , n_inputs , n_neurons):
    self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
    self.biases = np.zeros((1, n_neurons))

  def forward(self, inputs):
        # Save input for backward pass if needed
        self.inputs = inputs
        # Linear transformation: output = XW + b
        self.output = np.dot(inputs , self.weights) + self.biases

class Activation_Relu:
  def forward(self, inputs):
    self.output = np.maximum(0, inputs)


class Activation_Softmax:
  def forward(self, inputs):
    exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
    probabilities = exp_values / np.sum(exp_values, axis=1,keepdims=True)
    self.output = probabilities

class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        correct_confidences = y_pred_clipped[range(len(y_pred)), y_true]
        negative_log_likelihoods = -np.log(correct_confidences)
        return np.mean(negative_log_likelihoods)



In [42]:
X,y = vertical_data(samples = 1000 , classes=3)
layer1 = Dense_Layer(2,3)
activation1 = Activation_Relu()
layer1.forward(X)
activation1.forward(layer1.output)
layer2 = Dense_Layer(3,3)
activation2 = Activation_Softmax()
layer2.forward(activation1.output)
activation2.forward(layer2.output)
loss = Loss_CategoricalCrossentropy()
loss_value = loss.forward(activation2.output , y)
print(activation2.output)
print(loss_value)

[[0.33332306 0.33333775 0.33333918]
 [0.3333292  0.33333728 0.33333352]
 [0.33332056 0.33333486 0.33334458]
 ...
 [0.33329305 0.33333182 0.3333751 ]
 [0.3332952  0.33333156 0.33337325]
 [0.33329925 0.3333327  0.33336806]]
1.0985721


In [53]:
import torch
import torch.nn as nn
from nnfs.datasets import vertical_data
import nnfs

nnfs.init()

# --- Data ---
X, y = vertical_data(samples=1000, classes=3)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# --- Model ---
model = nn.Sequential(
    nn.Linear(2, 3),
    nn.ReLU(),
    nn.Linear(3, 3)  # logits
)

# --- Forward pass ---
logits = model(X)  # raw scores
softmax = nn.Softmax(dim=1)
probabilities = softmax(logits)  # convert logits to probabilities

# --- Compute Cross-Entropy manually ---
# Clip probabilities to avoid log(0)
probabilities = torch.clamp(probabilities, 1e-7, 1 - 1e-7)
# Gather the probability of the correct class for each sample
correct_probs = probabilities[range(len(y)), y]
# Negative log likelihood
loss = -torch.log(correct_probs)
# Mean loss over batch
loss = loss.mean()

print("Loss:", loss.item())
print("Probabilities (first 5 samples):")
print(probabilities[:5])

# --- Backward pass ---
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer.zero_grad()
loss.backward()  # PyTorch computes gradients automatically
optimizer.step()


Loss: 1.1635332107543945
Probabilities (first 5 samples):
tensor([[0.1981, 0.3486, 0.4533],
        [0.1976, 0.3463, 0.4561],
        [0.1987, 0.3401, 0.4612],
        [0.1985, 0.3474, 0.4541],
        [0.1984, 0.3469, 0.4547]], grad_fn=<SliceBackward0>)
